In [4]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
with open("ent2.txt", 'r') as file:
 FileContent = file.read().strip()

with open("ent2sum.txt", 'r') as file:
 FileContentSum = file.read().strip()

In [6]:
# display file content
FileContent

'Cary Fukunaga is a man of words.\n\nIt frustrates him if he cannot find the correct word to articulate a thought or a feeling he\'s had. I noticed this when filmmaker Zoya Akhtar interviewed him in a session held on the sidelines of the recently concluded 18th Mumbai Film Festival.\n\nThe American filmmaker was born in California to a father of Japanese origin and a Swedish-American mother. By his own admission, the multiculturalism at home was so deeply ingrained in him that it would eventually go on to help him write stories with characters that were easy to empathize with.\n\n"No doubt that a multicultural upbringing makes it very easier for you to access and understand stories of cultures far removed from your reality," Fukunaga said, addressing a question posed by a member in the audience.\n\n\n\nBefore finding his calling in artistic pursuits upon enrolling in a film school in NYU, Fukunaga aspired to be a professional snowboarder. "After film school, I was left with a debt of o

In [7]:
FileContentSum

"The American filmmaker was born in California to a father of Japanese origin and a Swedish-American mother. \n\nFukunaga's films have mostly been about displacement, lost identity, and the quest for belonging.\n\n\nHis debut, Sin Nombre, about a young boy and a girl trying to escape the hardships in Mexico by illegally immigrating to the US, won him an award for directing at the 2009 Sundance Film Festival, while Beasts of No Nation, another drama set in war-torn West Africa, also garnered interest at major award ceremonies.\n\nWhen you, as a white American director, go to conflict-ridden countries to try and tell their stories, there's a big risk that you take. It's easy for someone to label your work as 'poverty porn' or 'war porn' or to accuse you of being a know-it-all white savior trying to resolve issues on their behalf. I'm not like an issue person as such but if these are the realities of our times, our cinema should also capture and chronicle and make them the stories of our 

In [8]:
# total characters in the file
len(FileContent)

8792

In [9]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "facebook/bart-large-xsum"
#"knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM-AMI"
#"philschmid/bart-large-cnn-samsum"
#
#"facebook/bart-large-cnn"
# "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [10]:
# max tokens including the special tokens
tokenizer.model_max_length

1024

In [11]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [12]:
# number of special tokens
tokenizer.num_special_tokens_to_add()

2

In [13]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

69

In [15]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

2

In [16]:
[len(tokenizer.tokenize(c)) for c in chunks]

[993, 833]

In [17]:
[len(tokenizer(c).input_ids) for c in chunks]

[995, 835]

In [18]:
sum([len(tokenizer(c).input_ids) for c in chunks])

1830

In [19]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (1907 > 1024). Running this sequence through the model will result in indexing errors


1907

In [20]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

1826

In [21]:
len(tokenizer.tokenize(FileContent))

1905

In [22]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [23]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True,max_length=300,max_new_tokens=500))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 62 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The director of True Detective, Cary Fukunaga, is best known for directing the period drama Jane Eyre and the war drama Beasts of No Nation.
Ahead of the release of Beasts of No Nation and Sin Nombre, you were asked if your work sensationalises or eroticises the subject matter.


In [24]:
output_final =tokenizer.decode(*output, skip_special_tokens=True,max_length=2)

In [25]:
output_final

'Ahead of the release of Beasts of No Nation and Sin Nombre, you were asked if your work sensationalises or eroticises the subject matter.'

In [26]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from rouge import Rouge
ROUGE = Rouge()
ROUGE.get_scores(FileContentSum,output_final)

[{'rouge-1': {'r': 0.5714285714285714,
   'p': 0.08163265306122448,
   'f': 0.14285714066964286},
  'rouge-2': {'r': 0.21739130434782608,
   'p': 0.024630541871921183,
   'f': 0.044247785782363616},
  'rouge-l': {'r': 0.42857142857142855,
   'p': 0.061224489795918366,
   'f': 0.10714285495535718}}]